In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

import duckdb
def query(s):
    pd.options.display.max_rows = 50
    return duckdb.sql(s).df()

In [16]:

books_metadata = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "hadifariborzi/amazon-books-dataset-20k-books-727k-reviews",
  "amazon_books_metadata_sample_20k.csv",
)

books_reviews = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "hadifariborzi/amazon-books-dataset-20k-books-727k-reviews",
  "amazon_books_reviews_sample_20k.csv",
)


100%|██████████| 400M/400M [00:36<00:00, 11.5MB/s] 


In [2]:
# (Optional) Save the file locally 
import os

outdir = './dataset'
if not os.path.exists(outdir):
    os.mkdir(outdir)

books_metadata.to_csv('./dataset/metadata.csv',)
books_reviews.to_csv('./dataset/reviews.csv')


NameError: name 'books_metadata' is not defined

In [3]:
books_metadata = pd.read_csv('./dataset/metadata.csv', index_col=False)
books_reviews = pd.read_csv('./dataset/reviews.csv', index_col=False)


In [4]:
books_metadata.head()

,Unnamed: 0,title,subtitle,author_name,author_about,publisher,publisher_date,format,page_count,language,...,price_numeric,description,features_text,dimensions,item_weight,images,videos,store,parent_asin,bought_together
0,0,Government Bullies: How Everyday Americans are...,"Paperback – August 6, 2013",Rand Paul,"Discover more of the author’s books, see simil...",Center Street,"41260th edition (August 6, 2013",Paperback,320.0,English,...,12.99,"['Review', 'Praise for The Tea Party Goes to W...",Government regulations are out of control. The...,5.25 x 0.81 x 8 inches,9.2 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"Rand Paul (Author), Ron Paul (Foreword)",1455522775,NaN
1,1,Gordon Pope Thrillers: Books 1-3,"Paperback – December 17, 2020",B. B. Griffith,"B. B. Griffith was born and raised in Denver, ...",Griffith Publishing LLC,"December 17, 2020",Paperback,694.0,English,...,24.69,"['Review', 'Selected Reviews from the Gordon P...",Gordon Pope has a special talent. | He's the o...,6 x 1.74 x 9 inches,2.21 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],B. B. Griffith (Author),1735305855,NaN
2,2,Chart Sense: Common Sense Charts to Teach 3-8 ...,"Paperback – Illustrated, February 2, 2014",Rozlyn Linder,Follow Roz on Twitter (@rozlinder) and connect...,"Literacy Initiative, The","Illustrated edition (February 2, 2014",Paperback,182.0,English,...,22.88,"['Review', ""This is the first book I have ever...",Chart Sense is the ultimate resource for eleme...,7.5 x 0.41 x 9.25 inches,12 ounces,[],[],Rozlyn Linder (Author),0988950510,NaN
3,3,Where the Sky Begins: A Novel,"Paperback – August 2, 2022",Rhys Bowen,Rhys Bowen is the New York Times bestselling a...,Lake Union Publishing,"August 2, 2022",Paperback,399.0,English,...,6.15,"['Review', '“Bowen is a master at winning read...",A woman’s future is determined by fate and cho...,5.5 x 1 x 8.25 inches,14.1 ounces,[],[],Rhys Bowen (Author),1542028841,NaN
4,4,The Mystery of Garabandal: Fantasy or Fraud? G...,NaN,LR Walker,"Discover more of the author’s books, see simil...",Newtonia Publishing LLC,"First Paperback Edition (July 31, 2015",Paperback,299.0,English,...,12.99,"['About the Author', ""LR Walker studied journa...",This 2021 update includes new interviews with ...,6 x 0.68 x 9 inches,14.3 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"LR Walker (Author), Frank Lafleur (Introduction)",0692297677,NaN


In [54]:
pd.options.display.max_rows = None
pd.options.display.min_rows = None

# query("select right(publisher_date, 4) as year, count(distinct title) from books_metadata group by 1")
# query("select distinct right(publisher_date, 4) as year from books_metadata order by year")
processed_metadata = query("""
      
    with p1 as (
      select 
        * exclude(publisher_date),
        case 
          when try_cast(right(substr(publisher_date, strpos(publisher_date, '(')),4) as integer) is not null then substr(publisher_date, strpos(publisher_date, '(')+1) 
          else null
        end as date_str,
      from books_metadata 
      where parent_asin in ('0446529869', '194199962X')
    )
    
    select 
      *,
      strptime(date_str, '%B %-d, %Y') published_date
    from p1 
    """)
# The publisher date is like a scraping result from a dumb condition so we have to process our metadata to get a proper published_date column


In [ ]:
books_reviews.head()

,Unnamed: 0,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,date,year
0,0,5.0,Good choice,Just what I was looking for. Thank you.,[],1477493395,1477493395,AGXFEGMNVCSTSYYA5UWXDV7AFSXA,1529789386700,1,True,2018-06-23 21:29:46.700,2018
1,1,3.0,Perfect size,Very small lines to color had to get 4.0 marke...,[],1780679548,1780679548,AGPR2UDFIJJCBA3EKFEEAJBWKROQ,1591370837527,0,True,2020-06-05 15:27:17.527,2020
2,2,1.0,Much of this book was unbelievable and I almos...,Hard to get through was very disappointed with...,[],1785588451,1785588451,AEUH4EH6XHROLT7UZPUYU2YKTYMA,1618447014407,1,True,2021-04-15 00:36:54.407,2021
3,3,5.0,Great Buy!,"This series was requested by my grandson, who ...",[],1606412388,1606412388,AH6PLOGWYIVIWLJTY756BHNFD4YA,1340195874000,0,True,2012-06-20 12:37:54.000,2012
4,4,5.0,You can go home again!,"I love John Grogan, he has always been one of ...",[],0061713244,0061713244,AFSKPY37N3C43SOI5IEXEK5JSIYA,1224796701000,1,False,2008-10-23 21:18:21.000,2008


In [5]:
samplebook = query(
    """    
    select
        title, 
        publisher_date, 
        author_name,
        cast(trim(right(publisher_date,4)) as int64) published_year, 
        parent_asin 
    from books_metadata 
    where 1=1 
        and lower(author_name) like '%suzanne collins%'
    order by published_year desc
        limit 100
    """
)
p1 = query(
    """
select
      *
from books_reviews
where parent_asin in (select distinct parent_asin from samplebook)
-- limit 5      
"""
)

query("""
select parent_asin, count(1) from p1 group by 1 order by 2 desc
      """)

,parent_asin,count(1)
0,0439023521,1
1,2266182714,1
2,0545265355,1
3,1338321919,1
4,043979143X,1
5,0545422906,1


In [ ]:
select * from books_metadata
where lower()

,parent_asin,count(1)
0,0439023521,3150
1,0545265355,1827
2,0545422906,77
3,043979143X,17
4,2266182714,1
5,1338321919,1
